In [20]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from ml_utils.metrics import aur_pr
from answer_judge import load_judge

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
data_train=ReadData('/root/autodl-tmp/hallscope/AOKVQA/train').read_all()
random.shuffle(data_train)
#data_val=ReadData('/root/autodl-fs/hallscope/ScienceQA/val').read_all()
#data_test=ReadData('/root/autodl-fs/hallscope/ScienceQA/test').read_all()

In [22]:
df_train=pd.DataFrame(data_train)
#df_val=pd.DataFrame(data_val)
#df_test=pd.DataFrame(data_test)

In [23]:
df_train.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-tmp/coco2017/train2017/000000020...,[Question:Which animal would least like to be ...,"[tensor(3), tensor(0), tensor(2), tensor(3), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
1,[/root/autodl-tmp/coco2017/train2017/000000194...,[Question:The picture and outfit looks like it...,"[tensor(1), tensor(2), tensor(0), tensor(1), t...",{'embedding': [[[-0.00311279 0.00634766 0.04...,None
2,[/root/autodl-tmp/coco2017/train2017/000000482...,[Question:What is the woman using?\nAnswer the...,"[tensor(0), tensor(3), tensor(0), tensor(1), t...",{'embedding': [[[ 0.01013184 0.00717163 -0.00...,None
3,[/root/autodl-tmp/coco2017/train2017/000000568...,[Question:What brand is the motorcycle?\nAnswe...,"[tensor(0), tensor(3), tensor(2), tensor(0), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
4,[/root/autodl-tmp/coco2017/train2017/000000568...,[Question:What type of transportation is this?...,"[tensor(1), tensor(0), tensor(1), tensor(1), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None


In [24]:
i=0
df_train.iloc[i]['most_likely']['response']

['D', 'A', 'C', 'D.hole', 'C. donut', 'D.blue', 'D', 'A.stripes']

In [25]:
df_train.iloc[i]['img_path']

['/root/autodl-tmp/coco2017/train2017/000000020903.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000519188.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000570518.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000087792.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000230091.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000431494.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000310989.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000423810.jpg']

In [26]:
df_train.iloc[i]['question']

['Question:Which animal would least like to be in the sink if the faucet were turned on?\nAnswer the Question with following choices.\nChoices: A.reptile B.fish C.dog D.cat\n',
 'Question:What number is on the plane?\nAnswer the Question with following choices.\nChoices: A.410 B.369 C.821 D.775\n',
 'Question:Which object is most likely to be holding liquid right now?\nAnswer the Question with following choices.\nChoices: A.pot B.bowl C.bottle D.plate\n',
 'Question:What do half these treats have?\nAnswer the Question with following choices.\nChoices: A.chocolate chips B.sprinkles C.gummy bears D.hole\n',
 'Question:What treat is in the box?\nAnswer the Question with following choices.\nChoices: A.gummy bears B.donut C.pizza D.apple pie\n',
 'Question:What color is the item with the holes?\nAnswer the Question with following choices.\nChoices: A.purple B.red C.green D.blue\n',
 "Question:The nose of this aircraft is in what nation's flag?\nAnswer the Question with following choices.\nC

In [27]:
df_train.iloc[0]['answer']

tensor([3, 0, 2, 3, 1, 3, 2, 0])

In [28]:
embeddings=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(17056, 33, 4096)

In [29]:
gt_label=df_train['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(17056,)

In [30]:
gt_label[0]

np.int64(3)

In [31]:
judge=load_judge('OKVQA')

In [32]:
responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()

rsps=[]
for x in responses:
    rsps+=x
    
is_correct=[]
for i,x in enumerate(rsps):
    is_correct.append(judge.check(x,gt_label[i]))

is_correct=np.array(is_correct)
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 17056 responses. 11081 answer correctly and 5975 answer wrongly


In [33]:
length = len(embeddings)
train_rate = 0.8
num_val=2000

embedding_train = embeddings[:int(length*train_rate)-num_val]
embedding_val = embeddings[int(length*train_rate)-num_val:int(length*train_rate)]
embedding_test = embeddings[int(length*train_rate):]

is_correct_train = is_correct[:int(length*train_rate)-num_val]
is_correct_val = is_correct[int(length*train_rate)-num_val:int(length*train_rate)]
is_correct_test = is_correct[int(length*train_rate):]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(is_correct_train)}')
print(f'validset length:{len(is_correct_val)}')
print(f'testset length:{len(is_correct_test)}')
print(
    f'trainset postive: {sum(is_correct_train==1)} negative: {sum(is_correct_train==0)}')
print(
    f'valset postive: {sum(is_correct_val==1)} negative: {sum(is_correct_val==0)}')
print(
    f'testset postive: {sum(is_correct_test==1)} negative: {sum(is_correct_test==0)}')

trainset length:11644
validset length:2000
testset length:3412
trainset postive: 7542 negative: 4102
valset postive: 1294 negative: 706
testset postive: 2245 negative: 1167


In [34]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator,KernelPCA
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [35]:
M=4096
gamma=1

def evaluator(n_components,i_layer,X,y):
    kpca=PCADiscriminator(embedding_train[:,i_layer,:],n_components,kernel='poly')
    scores=kpca.get_score2(X[:,i_layer,:])
    return auroc(scores,y)

In [37]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(20,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,is_correct_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

found best n_components:1,layer:20 Score: 0.5366279757083248


  0%|          | 0/13 [00:00<?, ?it/s]

In [38]:
M=4096
gamma=1

def evaluator(n_components,i_layer,X,y):
    kpca=PCADiscriminator(embedding_train[:,i_layer,:],n_components,kernel='cosine')
    scores=kpca.get_score2(X[:,i_layer,:])
    return auroc(scores,y)

In [40]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,4),
    'layer':range(20,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,is_correct_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

found best n_components:1,layer:20 Score: 0.5310312140145628
found best n_components:1,layer:23 Score: 0.5331219268710239
found best n_components:1,layer:24 Score: 0.535032028407424
found best n_components:1,layer:25 Score: 0.5356734722471551


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [41]:
M=4096
gamma=1

def evaluator(n_components,i_layer,X,y):
    kpca=PCADiscriminator(embedding_train[:,i_layer,:],n_components,kernel='sigmoid')
    scores=kpca.get_score2(X[:,i_layer,:])
    return auroc(scores,y)

In [42]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(20,25)
}
grid_search=GridSearch(evaluator,grid,embedding_val,is_correct_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

found best n_components:1,layer:20 Score: 0.5396994627634188


  0%|          | 0/5 [00:00<?, ?it/s]

In [43]:
M=4096
gamma=1

def evaluator(n_components,i_layer,X,y):
    kpca=PCADiscriminator(embedding_train[:,i_layer,:],n_components,kernel='linear')
    scores=kpca.get_score2(X[:,i_layer,:])
    return auroc(scores,y)

In [44]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(20,25)
}
grid_search=GridSearch(evaluator,grid,embedding_val,is_correct_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

found best n_components:1,layer:20 Score: 0.5441999684751151


  0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
best_paras

{'n_components': 1, 'layer': 20, 'best_score': np.float64(0.5441999684751151)}

In [46]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [49]:
discriminator=PCADiscriminator(embedding_train[:,best_layer,:],best_paras['n_components'],kernel='linear')
scores=discriminator.get_score2(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,is_correct_val)

In [50]:
scores=discriminator.get_score2(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(is_correct_train,y_train)} AUROC:{roc_auc_score(is_correct_train,scores)} AUC-PR:{aur_pr(scores,is_correct_train)}')

train set acc: 0.45499828237719 AUROC:0.5422582990801649 AUC-PR:0.6841953141279782


In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
y_test=(scores>best_split)
print(f'test set acc: {accuracy_score(is_correct_test,y_test)} AUROC:{roc_auc_score(is_correct_test,scores)} AUC-PR:{aur_pr(scores,is_correct_test)}')

test set acc: 0.6099062133645955 AUROC:0.4978282783662886 AUC-PR:0.6561419856373895


Train Linear Prob to Get Better Results

In [23]:
X_train=embedding_train[:, best_layer,:]

X_test=embedding_test[:, best_layer,:]

In [24]:
best_acc, final_acc, (
            clf, best_state, best_preds, y_train, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
             y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [25]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores = torch.sigmoid(output).cpu().data.numpy()
y_preds=scores>0.5
print(f'test set acc: {accuracy_score(is_correct_test,y_preds)} AUROC:{roc_auc_score(is_correct_test,scores)} AUC-PR:{aur_pr(scores,is_correct_test)} f1:{f1_score(is_correct_test,y_preds)}')

test set acc: 0.6327667057444314 AUROC:0.47663800486983166 AUC-PR:0.6443647411016443 f1:0.7750852629689463
